In [1]:
import pandas as pd
import os
import numpy as np
from extract_audio_functions import * # importa como módulo completo
import importlib


# Leer el CSV
df = pd.read_csv("./dataset/spotify_songs_download.csv")

paths_audios_wav=df['path_download_wav'].tolist()
paths_audios_mp3=df['path_download'].tolist()


In [ ]:
from extract_audio_functions import *

# pasar de mp3 a wav
for i in range(0,len(paths_audios_wav)):
    entrad= paths_audios_mp3[i]
    salida = paths_audios_wav[i]
    if entrad.endswith(".mp3"):
        transform_mp3_to_wav(entrad,salida,tiempo_recorte=30)  # el path y el directorio de salida
    else:
        print(f"El archivo {entrad} no es un mp3, se omite la conversión.")


In [ ]:

# Obtener listas desde el DataFrame
paths_audios_wav = df["path_download_wav"].tolist()

mfcc_por_track = []
for path_wav in paths_audios_wav:
    mfcc = extract_mfcc(path_wav, ventana=0.5)  # Captura de 0.5 segundos
    mfcc_por_track.append( mfcc)



In [ ]:
mfcc_por_track[0].shape # cada audio

In [ ]:
#apilar los mfcc
mfcc_apilados= np.vstack(mfcc_por_track)
print(f"Forma de los MFCC apilados: {mfcc_apilados.shape}")

In [ ]:
#df.head()

In [ ]:

#NORMALIZAR LA DATA PARA EL KMEANS
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
mfcc_normalizado=scaler.fit_transform(mfcc_apilados)

# Sacamos lo Codeword   -> acoustic word
Kmeans =construir_codebook(mfcc_apilados,n_clusters=250)
#inertia : cuando se aumenta el K la inertia tiende a 0  : significa que tanto estan pegados los puntos. 

#save scaler y kmeans -> para procesar luego una query
guardar_objeto(scaler,"scaler.joblib")
guardar_objeto(Kmeans,"Kmeans.joblib")

In [ ]:
print(Kmeans.cluster_centers_.shape)  # (250, 13)
#print(Kmeans.cluster_centers_)        # valores de cada centroide


In [ ]:

track_ids = df['track_id'].tolist() 

diccionary={}
for i in range(0,len(mfcc_por_track)):
    mfcc = mfcc_por_track[i]

    #volvemos a normalizar 
    mfcc_scaled = scaler.transform(mfcc)

    hist = histogram_audio(mfcc_scaled, Kmeans)
    diccionary[track_ids[i]] = hist.tolist() #   para guardar track_id : histograma

    # (DEBUG) mostrar mapeo
    print(f"Audio {track_ids[i]} → {hist.tolist()}")

# Guardar en JSON
guardar_json(diccionary)


In [2]:
# TEST MUSIC
def process_audio_track_id(df,track_id,scaler):
    """
    Parametros:
        df: dataframe de audios con 'track_id'
        track_id : identificador del audio en el df
    """
    
    path = df[df['track_id'] == track_id]['path_download_wav'].values[0]
    mfcc_test = extract_mfcc(path)

    mfcc_normalizado_test = scaler.transform(mfcc_test) 

    return mfcc_normalizado_test
#rescatar los objetos
scaler=cargar_objeto("scaler.joblib")
kmeans_model=cargar_objeto("Kmeans.joblib")

# Normalizacion del test
mfcc_normalizado_test = process_audio_track_id(df=df,track_id='7258lBUFSE45SJmxoJmfYk',scaler=scaler) 
print(mfcc_normalizado_test)
hist = histogram_audio(mfcc_normalizado_test,kmeans_model= kmeans_model)

result = knn_lineal(hist,k=5)




[[-3.3623776e+00 -1.9968541e+00 -5.0892346e-02 -6.8288493e-01
  -2.8643182e-01 -1.5082993e-01  1.9971212e-02 -6.7805732e-03
   1.5917839e-01 -1.4394753e-02  1.5374796e-01  6.6777669e-02
   1.2188289e-01]
 [-1.0388023e+00  1.7726190e+00 -2.9260781e+00 -2.4330912e+00
  -3.5428304e-02 -2.3342693e+00 -1.7799884e+00 -1.6958700e+00
  -1.2176005e+00 -1.7883908e-03 -9.4520819e-01 -1.1310945e+00
  -7.2956741e-01]
 [-1.1325147e+00  1.6773858e+00 -2.7507989e+00 -2.9736636e+00
  -1.5187112e+00 -2.9934640e+00 -1.4296607e+00 -1.5268837e+00
  -8.4693354e-01  1.1263802e+00 -1.3015260e+00 -3.5421185e+00
  -1.6400347e+00]
 [-1.1096439e+00  1.4607430e+00 -2.6106682e+00 -2.4052184e+00
  -1.9586071e+00 -2.7237406e+00 -1.2982361e+00 -2.3322687e+00
  -9.6768618e-01  4.5911852e-01 -1.6834409e+00 -1.1572771e+00
  -7.6266050e-01]
 [-1.4580691e+00  1.7169478e+00 -1.1686558e+00 -1.8879631e+00
  -6.0866112e-01 -1.9668554e+00 -1.4328140e+00 -1.5333782e+00
  -1.0762073e+00 -3.1157494e-01 -1.9900504e+00 -2.3793845e+0

In [ ]:
result

In [ ]:
filas = df[df['track_id'].isin(result[0])]
# Agregar la columna 'score' con la distancia correspondiente
filas['score'] = filas['track_id'].map(result[1])
#print(filas)


In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

# Leer dataset original con géneros
df_genre = pd.read_csv('./dataset/spotify_songs_download_FINAL.csv')

# Ordenar por score
filas = filas.sort_values('score')

# Hacer el join por track_id
join_data = pd.merge(df_genre, filas, on='track_id', how='inner')

# Mostrar columnas deseadas
join_data.sort_values('score')[['score', 'track_artist_x', 'track_name_x', 'playlist_genre', 'playlist_subgenre', 'path_download_y']]


In [ ]:
# Calcular distancias a cada vector en el diccionario
import numpy as np

def distancia_euclidiana(val1, val2):
    return np.linalg.norm(np.array(val1) - np.array(val2))

dis = {}
for key, val in diccionary.items():  # <-- importante usar .items()
    dist = distancia_euclidiana(val, hist)
    dis[key] = dist

for track_id, distancia in sorted(dis.items(), key=lambda x: x[1]): #key : value
    print(f"{track_id}: {distancia:.4f}")


# Ordenar y obtener los 5 más cercanos
dis_sort = sorted(dis.items(), key=lambda x: x[1]) # orden descendente : si la distancia es menor es mas parecido
top_5_ids = [track_id for track_id, _ in dis_sort[:5]]

# Obtener filas correspondientes del DataFrame
filas = df[df['track_id'].isin(top_5_ids)]
# Agregar la columna 'score' con la distancia correspondiente
filas['score'] = filas['track_id'].map(dis)
#print(filas)


In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

# Leer dataset original con géneros
df_genre = pd.read_csv('./dataset/spotify_songs_download_FINAL.csv')

# Ordenar por score
filas = filas.sort_values('score')

# Hacer el join por track_id
join_data = pd.merge(df_genre, filas, on='track_id', how='inner')

# Mostrar columnas deseadas
join_data.sort_values('score')[['score', 'track_artist_x', 'track_name_x', 'playlist_genre', 'playlist_subgenre', 'path_download_y']]


In [ ]:
import subprocess

def play_audio(path):
    subprocess.call(['ffplay', '-nodisp', '-autoexit', path])

for i in filas['path_download_wav']:
    print(f"Reproduciendo: {i}")
    #play_audio(i)  # Reproducir cada audio
    
    #input("Presiona Enter para continuar...")  # Esperar a que el usuario presione Enter antes de continuar